In [21]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from matplotlib.colors import ListedColormap


# Custom colormap: yellow for empty, green for tree, red for burning
cmap = ListedColormap(['yellow', 'green', 'red'])

# Define the probabilities
probTree = 0.8       # Probability that a tree initially occupies a site
probBurning = 0.01   # Probability that the tree is initially burning

# Function to initialize the forest grid
def initialize_forest_grid(grid_size):
    grid = np.zeros((grid_size, grid_size), dtype=int)
    trees = np.random.rand(grid_size, grid_size) < probTree
    grid[trees] = 1
    burning_trees = (np.random.rand(grid_size, grid_size) < probBurning) & trees
    grid[burning_trees] = 2
    return grid


# Function to apply fire spread rules using von Neumann neighborhood with extended boundaries
def apply_fire_spread_rules_von_neumann(grid):
    grid_size = grid.shape[0]
    extended_grid_size = grid_size + 2
    # Create a temporary extended grid
    extended_grid = np.zeros((extended_grid_size, extended_grid_size), dtype=int)
    # Copy the original grid into the center of the extended grid
    extended_grid[1:-1, 1:-1] = grid
    # Apply periodic boundaries
    extended_grid[0, 1:-1] = grid[-1, :]   # Top row
    extended_grid[-1, 1:-1] = grid[0, :]   # Bottom row
    extended_grid[1:-1, 0] = grid[:, -1]   # Left column
    extended_grid[1:-1, -1] = grid[:, 0]   # Right column
    # Corners
    extended_grid[0, 0] = grid[-1, -1]
    extended_grid[-1, -1] = grid[0, 0]
    extended_grid[0, -1] = grid[-1, 0]
    extended_grid[-1, 0] = grid[0, -1]

    new_extended_grid = np.copy(extended_grid)
    for i in range(1, extended_grid_size - 1):
        for j in range(1, extended_grid_size - 1):
            if extended_grid[i, j] == 1:  # Tree
                for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:  # Von Neumann neighborhood
                    if extended_grid[i + di, j + dj] == 2:
                        new_extended_grid[i, j] = 2  # Catch fire
                        break
            elif extended_grid[i, j] == 2:  # Burning tree
                new_extended_grid[i, j] = 0  # Burn down

    # Extract the original grid size from the new extended grid, effectively removing the ghost cells
    new_grid = new_extended_grid[1:-1, 1:-1]

    return new_grid


# Function to run the simulation
def run_simulation(initial_grid, steps):
    states = [initial_grid]
    for _ in range(steps):
        new_grid = apply_fire_spread_rules_von_neumann(states[-1])
        states.append(new_grid)
    return states

def animate_simulation(states):
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_axis_off()
    ims = []
    for state in states:
        im = ax.imshow(state, cmap=cmap, animated=True, vmin=0, vmax=2)
        ims.append([im])
    ani = animation.ArtistAnimation(fig, ims, interval=200, repeat_delay=1000, blit=True)
    plt.close(fig)
    return HTML(ani.to_jshtml())
    
# Initialize the grid and run the simulation
grid_size = 100  # Define your grid size
initial_grid = initialize_forest_grid(grid_size)
states = run_simulation(initial_grid, 50)  # Run for 50 steps

# Animate and display the simulation
animation_html = animate_simulation(states)
animation_html


In [22]:
# step 2 


probImmune = 0.3     # Probability that a tree is immune to catching fire
probLightning = 0.001  # Probability of lightning strike

# Function to apply fire spread rules using the Moore neighborhood
def apply_fire_spread_rules_moore(grid, probImmune, probLightning):
    new_grid = np.copy(grid)
    grid_size = grid.shape[0]
    for i in range(grid_size):
        for j in range(grid_size):
            if grid[i, j] == 1:  # Tree
                # Moore neighborhood includes diagonal neighbors
                for di, dj in [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]:
                    ni, nj = (i + di) % grid_size, (j + dj) % grid_size  # Periodic boundary conditions
                    if grid[ni, nj] == 2:
                        if np.random.rand() > probImmune:
                            new_grid[i, j] = 2  # Tree catches fire
                        break
                if np.random.rand() < probLightning:
                    new_grid[i, j] = 2  # Tree catches fire due to lightning
            elif grid[i, j] == 2:  # Burning tree
                new_grid[i, j] = 0  # Burn down
    return new_grid

# Function to run the simulation with the Moore neighborhood
def run_simulation_moore(initial_grid, steps):
    states = [initial_grid]
    for _ in range(steps):
        new_grid = apply_fire_spread_rules_moore(states[-1], probImmune, probLightning)
        states.append(new_grid)
    return states


# Initialize the grid and run the simulation with the Moore neighborhood
states = run_simulation_moore(initial_grid, 50)  # Run for 50 steps

# Animate and display the simulation
animation_html = animate_simulation(states)
animation_html


In [23]:
from numba import jit, prange
import matplotlib.pyplot as plt
from IPython.display import HTML, display
import matplotlib.animation as animation


probImmune = 0.3     # Probability that a tree is immune to catching fire
probLightning = 0.001  # Probability of lightning strike

@jit(nopython=True, parallel=True)
def apply_fire_spread_rules_moore(grid, probImmune, probLightning):
    grid_size = grid.shape[0]
    extended_grid_size = grid_size + 2
    # Create a temporary extended grid with ghost cells
    extended_grid = np.zeros((extended_grid_size, extended_grid_size), dtype=grid.dtype)
    extended_grid[1:-1, 1:-1] = grid
    # Copy edges to ghost cells to simulate periodic boundary conditions
    extended_grid[0, 1:-1] = grid[-1, :]
    extended_grid[-1, 1:-1] = grid[0, :]
    extended_grid[1:-1, 0] = grid[:, -1]
    extended_grid[1:-1, -1] = grid[:, 0]
    # Corners
    extended_grid[0, 0] = grid[-1, -1]
    extended_grid[-1, -1] = grid[0, 0]
    extended_grid[0, -1] = grid[-1, 0]
    extended_grid[-1, 0] = grid[0, -1]

    new_grid = np.copy(grid)
    for i in prange(1, extended_grid_size - 1):
        for j in prange(1, extended_grid_size - 1):
            if extended_grid[i, j] == 1:  # Tree
                for di, dj in [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]:
                    if extended_grid[i + di, j + dj] == 2:
                        if np.random.rand() > probImmune:
                            new_grid[i-1, j-1] = 2  # Tree catches fire, accounting for ghost cell offset
                        break
                if np.random.rand() < probLightning:
                    new_grid[i-1, j-1] = 2  # Tree catches fire due to lightning, accounting for ghost cell offset
            elif extended_grid[i, j] == 2:  # Burning tree
                new_grid[i-1, j-1] = 0  # Burn down, accounting for ghost cell offset

    return new_grid

def run_simulation_parallel(initial_grid, steps):
    states = [initial_grid]
    for _ in range(steps):
        new_grid = apply_fire_spread_rules_moore(states[-1], probImmune, probLightning)
        states.append(new_grid)
    return states

states = run_simulation_parallel(initial_grid, 50)

# Animate and display the simulation
animation_html = animate_simulation(states)
display(animation_html)



In [24]:
#step 4 
import time

# Assume initialize_forest_grid, apply_fire_spread_rules_moore, run_simulation, and run_simulation_parallel are defined

# Initialize the grid
grid_size = 100
initial_grid = initialize_forest_grid(grid_size)

# Measure performance of the sequential simulation
start_time_seq = time.perf_counter()
states_seq = run_simulation_moore(initial_grid, 50)
end_time_seq = time.perf_counter()
execution_time_seq = end_time_seq - start_time_seq

# Measure performance of the parallel simulation
start_time_par = time.perf_counter()
states_par = run_simulation_parallel(initial_grid, 50)
end_time_par = time.perf_counter()
execution_time_par = end_time_par - start_time_par

# Compare and display the results
print(f"Sequential Execution Time: {execution_time_seq} seconds")
print(f"Parallel Execution Time: {execution_time_par} seconds")

# Optionally, you can visualize the final states to ensure both simulations produce similar results
# visualize_simulation(states_seq[-1])  # You might need to implement or adjust the visualization function
# visualize_simulation(states_par[-1])


Sequential Execution Time: 0.012167799985036254 seconds
Parallel Execution Time: 0.012346600065939128 seconds


In [25]:
#step 5

grid_sizes = [100, 200, 400, 800]  # Define your range of grid sizes
sequential_times = []
parallel_times = []

for grid_size in grid_sizes:
    initial_grid = initialize_forest_grid(grid_size)
    
    # Measure sequential execution time
    start_time = time.perf_counter()
    run_simulation_moore(initial_grid, 50)  # Adjust simulation parameters as needed
    sequential_times.append(time.perf_counter() - start_time)
    
    # Measure parallel execution time
    start_time = time.perf_counter()
    run_simulation_parallel(initial_grid, 50)
    parallel_times.append(time.perf_counter() - start_time)

# Compare and display the results
for i, grid_size in enumerate(grid_sizes):
    print(f"Grid Size: {grid_size}x{grid_size}")
    print(f"Sequential Execution Time: {sequential_times[i]} seconds")
    print(f"Parallel Execution Time: {parallel_times[i]} seconds")
    print("---")


Grid Size: 100x100
Sequential Execution Time: 0.01066030003130436 seconds
Parallel Execution Time: 0.010038800071924925 seconds
---
Grid Size: 200x200
Sequential Execution Time: 0.017448099912144244 seconds
Parallel Execution Time: 0.021892499993555248 seconds
---
Grid Size: 400x400
Sequential Execution Time: 0.03572020004503429 seconds
Parallel Execution Time: 0.026657599955797195 seconds
---
Grid Size: 800x800
Sequential Execution Time: 0.11323629994876683 seconds
Parallel Execution Time: 0.10997900005895644 seconds
---
